<h1>Welcome to CAP Notebooks!&emsp;<img style="vertical-align:middle" src="https://cap.cloud.sap/docs/assets/logos/sap.svg" alt="drawing" width="70"/><img style="vertical-align:middle" src="https://cap.cloud.sap/docs/assets/logos/cap.svg" alt="drawing" width="60"/></h1>

## What is a CAP Notebook?

A CAP notebook is a [Jupyter Notebook](https://jupyter.org/) that serves as a guide on how to create, navigate and monitor CAP projects.

With this, we want to encourage the CAP community to work with CAP in the same *explorative* manner that data scientists work with their data by:

- Visually interacting with their code
- Playing with REPL-type inputs (notebook input cells)
- Storing persistent code (notebook output cells)

The cell inputs/outputs are especially useful at later points in time when the project's details have long been forgotten. In addition, notebooks are a good way to share, compare and also reproduce projects.
<br><br>

## How do I run the CAP Notebook?

The following symbols indicate the state of a cell:

&emsp;&emsp;&emsp;  `▶` : Executes the cell

&emsp;&emsp;&emsp;  `*`   : Indicates that the cell is *running*
  
&emsp;&emsp;&emsp;  `■`   : Stops cell execution

When finished running, you will see the cell's output appear below the code cell. The blue bar on the left moves to keep track of where you are in the notebook. You can use `+` to add new cells anywhere in the notebook. See [Guide](https://code.visualstudio.com/docs/python/jupyter-support#_create-or-open-a-jupyter-notebook) for a more detailed description.
<br><br>

## How do I interact with the CAP Notebook?

In notebooks, predefined [Magic commands](https://ipython.readthedocs.io/en/stable/interactive/magics.html) have been designed to succinctly solve various common problems. They all start with `%` or `%%` for line or cell magic commands.

&emsp;&emsp;&emsp; `%quickref`  : Provides a full list of magic functions available

You will find these being used throughout the notebook.
<br><br>

## How do I get started?

Start by executing the first cell, which registers some additional magic commands which make our notebook platform-independent (across Windows, Linux and Mac).

After that, you're ready to start the CAP [Getting started in a Nutshell](https://cap.cloud.sap/docs/get-started/in-a-nutshell) tutorial. 

**```🡇 NOTE: This cell must be included and run at the start of any CAP Notebook! 🡇```**

In [ ]:
import glob, json
with open(glob.glob('./*.ipynb')[0], encoding='utf8') as json_file:
    exec(json.load(json_file)['metadata']['cap_magic'])

# Getting Started in a Nutshell
## Using a minimalistic setup


Get started in a minimalistic setup with *[Node.js](https://nodejs.org)*, *[express](http://expressjs.com)* and *[SQLite](http://sqlite.org)*.

 See also: *[Java-based variant of this guide](https://cap.cloud.sap/docs/java/getting-started)*.






## Jumpstarting Projects<a id="start-a-project"></a>


[After installing `@sap/cds-dk` globally](https://cap.cloud.sap/docs/get-started/#local-setup), `cds init` has already created a project with minimal defaults for us.

### Launch `cds watch`


For an automated jumpstart, you can just tell `cds` to watch out for things to arrive:



In [ ]:
!cds watch

Use `cds watch` to start a cds server, even in a newly created and yet empty project. Whenever you feed your project with new content, for example, by adding or modifying `.cds`, `.json`, or `.js` files, the server automatically restarts to serve the new content. Because there isn’t any content in your project yet, it just keeps waiting with a message like this:

<pre><code>
[cds] - running nodemon...
--ext cds,csn,csv,ts,mjs,cjs,js,json,properties,edmx,xml

    No models found at db/,srv/,app/,schema,services,.
    Waiting for some to arrive...

</pre></code>


## Defining Domain Models<a id="domain-models"></a>



Let’s feed our project by adding a simple domain model. Start by creating a file named `db/schema.cds` (also indicated in the code box’s label) and copy the following definitions into it:

In [ ]:
%%writefile "db/schema.cds"

using { Currency, managed, sap } from '@sap/cds/common';
namespace sap.capire.bookshop;

entity Books : managed {
  key ID : Integer;
  title  : localized String(111);
  descr  : localized String(1111);
  author : Association to Authors;
  genre  : Association to Genres;
  stock  : Integer;
  price  : Decimal(9,2);
  currency : Currency;
}

entity Authors : managed {
  key ID : Integer;
  name   : String(111);
  books  : Association to many Books on books.author = $self;
}

/** Hierarchically organized Code List for Genres */
entity Genres : sap.common.CodeList {
  key ID   : Integer;
  parent   : Association to Genres;
  children : Composition of many Genres on children.parent = $self;
}

⮡   [Find this source also in **cap/samples**](https://github.com/sap-samples/cloud-cap-samples/tree/master/bookshop/db/schema.cds)



  ⮡   [Learn more about **Domain Modeling**](https://cap.cloud.sap/docs/guides/domain-models)



  ⮡   [Learn more about **CDS Modeling Languages**](https://cap.cloud.sap/docs/cds)



### Deployed to Databases Automatically


As soon as you save your file, the still running `cds watch` will react immediately with new output like this:

<pre><code>
[cds] - connect to db { database: *':memory:'* }
/> successfully deployed to sqlite in-memory db
</pre></code>


This means that `cds watch` detected the changes in `db/schema.cds` and automatically bootstrapped an in-memory SQLite database when restarting the server process.


  ⮡   [Learn more about using databases below](https://cap.cloud.sap/docs/get-started/in-a-nutshell#databases)





## Defining Services<a id="defining-services"></a>



After the recent changes, `cds watch` also prints this message:

<pre><code>
    No service definitions found in loaded models.
    Waiting for some to be added...
</pre></code>


So, let’s go on feeding it with service definitions.
Following the [best practice of single-purposed services](https://cap.cloud.sap/docs/guides/services#single-purposed-services), we’ll define two services for different use cases.

### One for Admins to Maintain *Books* and *Authors*


In [ ]:
%%writefile "srv/admin-service.cds"

using { sap.capire.bookshop as my } from '../db/schema';
service AdminService @(_requires:'authenticated-user') {
  entity Books as projection on my.Books;
  entity Authors as projection on my.Authors;
}

⮡   [Find this source also in **cap/samples**](https://github.com/sap-samples/cloud-cap-samples/tree/master/bookshop/srv/admin-service.cds)



### And One for End Users to Browse and Order *Books*


In [ ]:
%%writefile "srv/cat-service.cds"

using { sap.capire.bookshop as my } from '../db/schema';
service CatalogService @(path:'/browse') {

  @readonly entity Books as SELECT from my.Books {*,
    author.name as author
  } excluding { createdBy, modifiedBy };

  @requires_: 'authenticated-user'
  action submitOrder (book: Books:ID, amount: Integer);
}

⮡   [Find this source also in **cap/samples**](https://github.com/sap-samples/cloud-cap-samples/tree/master/bookshop/srv/cat-service.cds)



  ⮡   [Learn more about **Defining Services**](https://cap.cloud.sap/docs/guides/services)



### Served to OData out-of-the-box


This time `cds watch` reacted with additional output like this:

<pre><code>
[cds] - serving AdminService { at: *'/admin'* }
[cds] - serving CatalogService { at: *'/browse'*, impl: *'bookshop/srv/cat-service.js'* }
[cds] - launched in: 744.291ms
[cds] - server listening on { url: *'http://localhost:4004'* }
</pre></code>


As you can see in the log output, the two service definitions have been compiled and generic service providers have been constructed to serve requests on the listed endpoints `/admin` and `/browse`.

Open *[http://localhost:4004](http://localhost:4004)* in your browser and see the generic `index.html` page:



In [ ]:
%%html
<iframe src='http://localhost:4004' width=1000 height=600 style='background-color:white'></iframe>

## Using Databases<a id="databases"></a>


### Using *sqlite* In-Memory Database


As [previously shown](https://cap.cloud.sap/docs/get-started/in-a-nutshell#deployed-in-memory), `cds watch` automatically bootstraps an SQLite in-process and in-memory database by default — that is, unless told otherwise. While this **isn’t meant for productive use**, it drastically speeds up development turn-around times, essentially by mocking your target database, for example, SAP HANA.


  ⮡   [Learn more about mocking options in **Grow as you go**](https://cap.cloud.sap/docs/get-started/grow-as-you-go)



### Adding Initial Data in `.csv` Files


Now, let’s fill your database with initial data by adding a few plain CSV files under `db/data` like this:

In [ ]:
%cd db
%mkdir data
%cd ..

In [ ]:
%%writefile "db/data/sap.capire.bookshop-Books.csv"

ID;title;author_ID;stock
201;Wuthering Heights;101;12
207;Jane Eyre;107;11
251;The Raven;150;333
252;Eleonora;150;555
271;Catweazle;170;22

In [ ]:
%%writefile "db/data/sap.capire.bookshop-Authors.csv"

ID;name
101;Emily Brontë
107;Charlotte Brontë
150;Edgar Allen Poe
170;Richard Carpenter

⮡   [Find a full set of `.csv` files in **cap/samples**](https://github.com/sap-samples/cloud-cap-samples/tree/master/bookshop/db/data)





After you’ve added these files, `cds watch` restarts the server with output, telling us that the files have been detected and their content been loaded into the database automatically:

<pre><code>
[cds] - connect to db { database: *':memory:'* }
 > filling sap.capire.bookshop.Authors from bookshop/db/data/sap.capire.bookshop-Authors.csv
 > filling sap.capire.bookshop.Books from bookshop/db/data/sap.capire.bookshop-Books.csv
/> successfully deployed to sqlite in-memory db
</pre></code>
> This is the output when you’re using the [samples](https://github.com/sap-samples/cloud-cap-samples). It’s less if you’ve followed the manual steps here.




  ⮡   [Learn more about **Using Databases**](https://cap.cloud.sap/docs/guides/databases)



### Querying Through OData out-of-the-box


Now that we’ve a connected, fully capable SQL database, filled with some initial data, we can send complex OData queries, served by the built-in generic providers:


  
- *[browse/Books?\$select=ID,title](http://localhost:4004/browse/Books?\$select=ID,title)*

  
- *[admin/Authors?\$search=Bro](http://localhost:4004/admin/Authors?\$search=Bro)*

  
- *[admin/Authors?\$expand=books(\$select=ID,title)](http://localhost:4004/admin/Authors?\$expand=books(\$select=ID,title))*




  ⮡   [Learn more about **Generic Providers**](https://cap.cloud.sap/docs/guides/generic)



  ⮡   [Learn more about **OData’s Query Options**](https://cap.cloud.sap/docs/advanced/odata)





## Summary and Next Steps…<a id="summary-and-next-steps"></a>




[**Visit our Cookbook**](https://cap.cloud.sap/docs/guides/) to find more task-oriented guides. For example, you can find guides about potential next steps such as adding [Authentication](https://cap.cloud.sap/docs/node.js/authentication) and [Authorization](https://cap.cloud.sap/docs/guides/authorization) or [Deploying to Cloud Foundry](https://cap.cloud.sap/docs/advanced/deploy-to-cloud).

Also **see the reference sections** to find detailed documentation about [**CDS**](https://cap.cloud.sap/docs/cds), as well as [**Node.js**](https://cap.cloud.sap/docs/node.js) and [**Java**](https://cap.cloud.sap/docs/java) Service SDKs and runtimes.